# Задание
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД

# Задание *
* Написать скрипт, который скачивает фотографии всех строящихся объектов с сайта "наш.дом.рф"


In [1]:
import requests
import pandas as pd

## Список объектов

Алексей, я установил лимит 500 записей. При этом блок по созданию датафрейма у меня работает около получаса.
При установке значения 1000 ноутбук зависает. Возможно, не хватает мощностей на моем конечном устройстве.
Делаю в Ананконде, если будет время - попробую переделать в Колабе, используя мощности гугл. Но пока так.

In [2]:
offset_ = 2001
limit_ = 500
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
res = requests.get(url)
objects_data = res.json()

In [3]:
objects_data.keys()

dict_keys(['data', 'errcode'])

In [4]:
objects_data.get('data').keys()

dict_keys(['list', 'total'])

In [5]:
objects_list = objects_data.get('data').get('list')

Я не придумал, как отсечь объекты по статусу, если использовать ваш вариант:

objids = [x.get('objId') for x in objects_list]

Пришлось перебирать по одиночке, так же прошу прокомментировать - есть ли тут возможность более красивого решения?

In [6]:
objids = []
for x in objects_list:
    status = x.get('siteStatus')
    if status == 'Строящиеся':
        objids.append(x.get('objId'))

# Данные по определенному объекту

In [7]:
df = pd.DataFrame()

Вот этот блок работает крайне медленно:

In [8]:
for obj_id in objids:
    url1 = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{obj_id}'
    res = requests.get(url1)
    obj1=pd.json_normalize(res.json())
    df = df.append(obj1)

Выгрузка получившегося датафрейма - тут все просто

In [9]:
df.to_excel("output.xlsx")

In [10]:
df.to_pickle("output.pkl")

In [11]:
import sqlite3

In [12]:
conn = sqlite3.connect('test_database')

А вот эту ошибку я победить не смог. Если есть возможность - прокомментируйте, пожалуйста.

In [13]:
df.to_sql('t_Objects',conn, if_exists='replace', index=False)

InterfaceError: Error binding parameter 50 - probably unsupported type.

In [14]:
conn.commit()
conn.close()